# How to implement gridsearch 

In [16]:
TRAIN_PATH = 'porto_data/train.csv'
TEST_PATH = 'porto_data/test.csv'
RESULT_FILE_NAME = 'grid_search_result.txt'

In [14]:
from __future__ import division
#import required modules
from datetime import datetime
import pandas as pd 
from IPython.display import display
import numpy as np

import xgboost as xgb
import time

from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

## load and processs data for grid search 

In [10]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

In [11]:
id_test = test['id'].values
id_train = train['id'].values
y = train['target']

In [12]:
### Drop calc
unwanted = train.columns[train.columns.str.startswith('ps_calc_')]
train.drop(unwanted, axis=1, inplace=True)  
test.drop(unwanted, axis=1, inplace=True)

In [13]:
#model
xgb_gsearch_params = {'max_depth':[4,5]}
xgb_model = XGBClassifier(    
                        n_estimators=400,
                        objective="binary:logistic",
                        learning_rate=0.07, 
                        subsample=.7,
                        colsample_bytree=.8,
                        scale_pos_weight=1.6,
                        gamma=10,
                        reg_alpha=8,
                        reg_lambda=1.4,
                            nthread=1, min_child_weight=7
                     )

In [ ]:
folds = 5
param_comb = 2

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb_model, param_distributions=xgb_gsearch_params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(train_oh,target), verbose=3)

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(train_oh, target)
timer(start_time) # timing ends here for "start_time" variable

### save result in a file 

In [ ]:
f = open(RESULT_FILE_NAME, 'w')
f.write('\n Best normalized gini score for {}-fold search with {} parameter combinations:'.format(str(folds), str(param_comb)))
f.write('Best normalized gini score:\n')
f.write(str(random_search.best_score_ * 2 - 1))
f.write('\n Best hyperparameters:')
f.write(str(random_search.best_params_))
f.write('\n model parameters:')
f.write(str(xgb_model))
f.close()

### Print Result in File 

In [ ]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)